In [1]:
import pandas as pd
import json

pairs = {eval(k): v for k,v in dict(json.loads(open("data/info/pairs.json", "r").read())).items()}

pair = ('ADX', 'TY')
df_A = pd.read_pickle("data/raw/" + pair[0] + ".zip")  # needs pickle5 compression (python 3.8)
df_B = pd.read_pickle("data/raw/" + pair[1] + ".zip")  # needs pickle5 compression (python 3.8)

pair_df = generate_pair_data(df_A, df_B, "1999-01-01", "2021-03-01")

pair_df.loc["2000-01-01":, :]

for pair in pairs.keys():
    df_A = pd.read_pickle("data/raw/" + pair[0] + ".zip")  # needs pickle5 compression (python 3.8)
    df_B = pd.read_pickle("data/raw/" + pair[1] + ".zip")  # needs pickle5 compression (python 3.8)
    pair_df = generate_pair_data(df_A, df_B, "1999-01-01", "2021-03-01")
    pair_df.to_pickle(f"data/processed/{pair[0].upper()}-{pair[1].upper()}.zip")
    print(f"saved data/processed/{pair[0].upper()}-{pair[1].upper()}.zip")

{('ADX', 'TY'): 0.853938041834032,
 ('ED', 'SO'): 0.7630684994843583,
 ('AEG', 'ING'): 0.7714520036919487,
 ('GOLD', 'NEM'): 0.8219942702537909,
 ('AVB', 'EQR'): 0.8793959812778221,
 ('AMAT', 'KLAC'): 0.8120695844768887,
 ('APA', 'DVN'): 0.7797792484401138,
 ('ARW', 'AVT'): 0.7573572791543869,
 ('CMA', 'TFC'): 0.8089975734507149,
 ('GFI', 'HMY'): 0.7721582544936532,
 ('BAC', 'WFC'): 0.8033690173669585,
 ('BBVA', 'SAN'): 0.8696850470469416,
 ('HIW', 'KRC'): 0.8127703586697915,
 ('BEN', 'TROW'): 0.7533682263460231,
 ('BK', 'NTRS'): 0.7853258900728711,
 ('HAL', 'SLB'): 0.7707796197283248,
 ('BMO', 'RY'): 0.8112242300303579,
 ('E', 'TOT'): 0.8145755214453303,
 ('BXP', 'VNO'): 0.8658210823728937,
 ('CCL', 'RCL'): 0.7796060510803956,
 ('CM', 'TD'): 0.7955914439701682,
 ('CVX', 'XOM'): 0.8257217118360992,
 ('CPT', 'UDR'): 0.8316230952071983,
 ('CSX', 'NSC'): 0.7742776958878398,
 ('CUZ', 'WRE'): 0.7571898754103825,
 ('DHI', 'LEN'): 0.8014581824672958,
 ('DRE', 'PLD'): 0.8245693410742715,
 ('EL

saved ('ADX', 'TY') data to data/processed/ADX-TY.zip
saved ('ED', 'SO') data to data/processed/ED-SO.zip
saved ('AEG', 'ING') data to data/processed/AEG-ING.zip
saved ('GOLD', 'NEM') data to data/processed/GOLD-NEM.zip
saved ('AVB', 'EQR') data to data/processed/AVB-EQR.zip
saved ('AMAT', 'KLAC') data to data/processed/AMAT-KLAC.zip
saved ('APA', 'DVN') data to data/processed/APA-DVN.zip
saved ('ARW', 'AVT') data to data/processed/ARW-AVT.zip
saved ('CMA', 'TFC') data to data/processed/CMA-TFC.zip
saved ('GFI', 'HMY') data to data/processed/GFI-HMY.zip
saved ('BAC', 'WFC') data to data/processed/BAC-WFC.zip
saved ('BBVA', 'SAN') data to data/processed/BBVA-SAN.zip
saved ('HIW', 'KRC') data to data/processed/HIW-KRC.zip
saved ('BEN', 'TROW') data to data/processed/BEN-TROW.zip
saved ('BK', 'NTRS') data to data/processed/BK-NTRS.zip
saved ('HAL', 'SLB') data to data/processed/HAL-SLB.zip
saved ('BMO', 'RY') data to data/processed/BMO-RY.zip
saved ('E', 'TOT') data to data/processed/E-TO

In [ ]:
model = ElasticNet()
model.fit(np.array(pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)")),
          np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"]))
pred_diff = model.predict(np.array(pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)")))
len(pred_diff)

In [ ]:
len(np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"]))

In [ ]:
print(np.std(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.mean(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.min(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.max(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
mean_squared_error(np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"]), pred_diff)

In [ ]:
annualized_return(df_A.loc[:,["Simple Return", "Log Return"]])

In [ ]:
from utils.models import GeometricBrownianMotion

GeometricBrownianMotion(1, 0.03, 0.05).simulate()